In [51]:
from datetime import datetime
from typing import Union

def remove_extra_last_char_(original_string, char=":", replace_char=""):
    last_char_index = original_string.rfind(char)
    return original_string[:last_char_index] + replace_char + original_string[last_char_index+1:]


class DateType(datetime):
    """
        L'indicateur de fuseau horaire Z ne doit pas être utilisé. Le fuseau horaire pour UTC doit être représenté par '-00:00'.

        ...

        Attributes
        ----------
        value : str
            a datetime object "\\d\\d\\d\\d-\\d\\d-\\d\\dT\\d\\d:\\d\\d:\\d\\d[\\-+]\\d\\d:\\d\\d"
            example 2070-11-02T16:05:29+00:00

    """

    def __new__(cls, *args, **kwargs):
        value = args[0]
        if isinstance(value, str):
            return datetime.strptime(remove_extra_last_char_(value), '%Y-%m-%dT%H:%M:%S%z')
        else:
            return value

    def __repr__(self):
        string_value = self.value.astimezone(timezone.utc).strftime('%Y-%m-%dT%H:%M:%S%z')
        return string_value[0:-2] + ":" + string_value[-2:]

    def __str__(self):
        return self.__repr__()

    def _to_dict(self):
        return self.__repr__()

In [53]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json

@dataclass_json
@dataclass
class B:
    a:DateType
        
    class Config:
        arbitrary_types_allowed = True


In [57]:
b = B(a=DateType(datetime.now()))

In [58]:
import json

In [59]:
b.to_dict()

{'a': datetime.datetime(2020, 11, 12, 11, 52, 4, 171493)}

In [1]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json

In [23]:
from datetime import datetime
from dataclasses import dataclass, field
from dataclasses_json import dataclass_json
from typing import List, Dict


@dataclass_json
@dataclass
class ContactMethods:
    tel: Dict[str, str]
    email: str
    address: str


@dataclass
@dataclass_json
class ContactEntity:
    first_name: str
    last_name: str
    position: str
    group_name: str
    creator_id: str
    contact_methods: ContactMethods = field(default_factory=ContactMethods)
    groups: List = field(default_factory=lambda: [])
    created_at: datetime = field(default_factory=lambda: datetime.utcnow())
    updated_at: datetime = field(default_factory=lambda: datetime.utcnow())



In [24]:
ContactMethods.from_dict({
    "address": "8 av du docteur blanchet, 77500 Chelles",
    "email": "r.courivaud@gmail.com",
    "tel": {
      "mobile": "0615409041"
    }
  })

ContactMethods(tel={'mobile': '0615409041'}, email='r.courivaud@gmail.com', address='8 av du docteur blanchet, 77500 Chelles')

In [29]:
data = {
  "contact_methods": {
    "address": "8 av du docteur blanchet, 77500 Chelles",
    "email": "r.courivaud@gmail.com",
    "tel": {
      "mobile": "0615409041"
    }
  },
  "first_name": "Raphaël",
  "group_name": "Mairie Chelles",
  "last_name": "Courivaud",
  "position": "maire"
}
data["creator_id"] = "my_id"
ContactEntity.from_dict(data)

TypeError: argument of type 'NoneType' is not iterable

TypeError: object() takes no parameters

In [3]:
import sys
print(sys.version)

3.6.10 |Anaconda custom (64-bit)| (default, Mar 23 2020, 17:45:12) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [ ]:
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict
from dataclasses_json import dataclass_json, DataClassJsonMixin


@dataclass
class ContactMethods(DataClassJsonMixin):
    tel: Dict[str, str]
    email: str
    address: str


@dataclass
class ContactEntity(DataClassJsonMixin):
    first_name: str
    last_name: str
    position: str
    group_name: str
    creator_id: str
    contact_methods: ContactMethods = field(default_factory=ContactMethods)
    groups: List = field(default_factory=lambda: [])
    created_at: datetime = field(default_factory=lambda: datetime.utcnow())
    updated_at: datetime = field(default_factory=lambda: datetime.utcnow())


cm = ContactMethods.from_dict({
    "address": "8 av du docteur blanchet, 77500 Chelles",
    "email": "r.courivaud@gmail.com",
    "tel": {
      "mobile": "0615409041"
    }
  })

In [2]:

print(cm)

data = {"contact_methods": {
    "address": "8 av du docteur blanchet, 77500 Chelles",
    "email": "r.courivaud@gmail.com",
    "tel": {
        "mobile": "0615409041"
    }
}, "first_name": "Raphaël", "group_name": "Mairie Chelles", "last_name": "Courivaud", "position": "maire",
    "creator_id": "my_id"}

print(ContactEntity.from_dict(data))

ContactMethods(tel={'mobile': '0615409041'}, email='r.courivaud@gmail.com', address='8 av du docteur blanchet, 77500 Chelles')


TypeError: argument of type 'NoneType' is not iterable

In [9]:
from marshmallow import Schema, fields, post_load


In [25]:
class Test(Schema):
    contact_methods = fields.Method("get_contact_methods", deserialize="load_contact_methods")

    def get_contact_methods(self, obj):
        return obj

    def load_contact_methods(self, value):
        return ContactMethods.from_dict(value)


In [26]:
t = Test().load({
    "contact_methods":{
    "address": "8 av du docteur blanchet, 77500 Chelles",
    "email": "r.courivaud@gmail.com",
    "tel": {
      "mobile": "0615409041"
    }
  }        
    })

In [27]:
t

{'contact_methods': ContactMethods(tel={'mobile': '0615409041'}, email='r.courivaud@gmail.com', address='8 av du docteur blanchet, 77500 Chelles')}

In [28]:
Test().dump(t)

{'contact_methods': {'contact_methods': ContactMethods(tel={'mobile': '0615409041'}, email='r.courivaud@gmail.com', address='8 av du docteur blanchet, 77500 Chelles')}}